In [1]:
from types import MethodType
from typing import Any
from multiprocessing import Pool
from statistics import mode as stats_mode
from pathlib import Path
import os

import dill
import laspy
import CSF
import numpy as np
import pandas as pd 
import open3d as o3d
from scipy.spatial import KDTree
from scipy.stats import pointbiserialr, mode as scipy_mode
import matplotlib.pyplot as plt
from tqdm import tqdm

import analysis_utils

In [2]:
PROCESSES = 8

LIDAR_CLASSIFICATION_CODES = {
    0: 'never classified',
    1: 'unassigned',
    2: 'ground',
    3: 'low vegetation',
    4: 'medium vegetation',
    5: 'high vegetation',
    6: 'building',
    7: 'noise',
    8: 'model key/reserved',
    9: 'water',
    10: 'rail',
    11: 'road surface',
    12: 'overlap/reserved',
    13: 'wire - guard',
    14: 'wire - conductor',
    15: 'transmission tower',
    16: 'wire - connector',
    17: 'bridge deck',
    18: 'high noise',
}

In [3]:
# Data: https://opendata.vancouver.ca/explore/dataset/lidar-2018/information/

# # Kits
# data_folder = Path('./data/4870E_54560N_kits')
# las_file_path = data_folder / '4870E_54560N.las'

# Point Grey forest + buildings
data_folder = Path('./data/483000_5457000_ptgrey')
las_file_path = data_folder / '483000_5457000.las'

# # Downtown buildings
# data_folder = Path('./data/491000_5458000_downtown')
# las_file_path = data_folder / '491000_5458000.las'

las_file = laspy.read(las_file_path)

### Define a class to make the las data easier to manipulate

In [4]:
class LasPointData:
    def __init__(self, las: laspy.LasData) -> None:
        self.las = las
        self.header = las.header
        self.vlrs = las.vlrs
        self.point_format = las.point_format
        self.point_count = las.header.point_count

        self.point_dimensions = ['x', 'y', 'z'] + [d.name for d in las.point_format.dimensions]

        self.data = {}
        for dim in self.point_dimensions:
            self.data[dim] = np.asarray(getattr(las.points, dim))
        
    def header_info(self) -> None:
        for attr in dir(self.header):
            if attr.startswith('_') or attr.isupper():
                continue
            if isinstance(val := getattr(self.header, attr), MethodType):
                continue
            print(f'{attr}: {val}')
    
    def crs(self) -> None:
        for vlr in self.vlrs:
            try:
                print(repr(vlr.parse_crs()))
            except AttributeError:
                pass
    
    def get_point_format(self) -> tuple[laspy.PointFormat, list[str]]:
        return self.point_format, self.point_dimensions
    
    def get_point_data(self, scaled: bool = True, step: int = 1, normalized: bool = False) -> np.ndarray:
        if normalized and not scaled:
            raise ValueError('Unscaled data cannot be normalized')
        if normalized and not hasattr(self, 'z_normalized'):
            raise AttributeError(f'Attribute \'z_normalized\' does not exist')

        x = self.x if scaled else self.X
        y = self.y if scaled else self.Y
        z = self.data['z_normalized' if normalized else 'z'] if scaled else self.Z
        data = np.vstack((x, y, z)).T

        if step > 1:
            mask = np.zeros(self.point_count, dtype=bool)
            mask[::step] = True
            return data[mask]
        return data
    
    def get_class_names(self) -> np.ndarray:
        lcc_get_partial = lambda code: LIDAR_CLASSIFICATION_CODES.get(code, '')
        return np.vectorize(lcc_get_partial)(self.classification)
    
    def get_extent(self, normalized: bool = False) -> tuple[float, ...]:
        if normalized:
            if not hasattr(self, 'z_normalized'):
                raise AttributeError(f'Attribute \'z_normalized\' does not exist')
            return self.x.min(), self.x.max(), self.y.min(), self.y.max(), self.z_normalized.min(), self.z_normalized.max()
        return self.x.min(), self.x.max(), self.y.min(), self.y.max(), self.z.min(), self.z.max()
    

    def apply_mask(self, mask: np.ndarray) -> None:
        assert isinstance(mask, np.ndarray) and mask.dtype == bool, '\'mask\' must be a numpy.ndarray with dtype bool'
        
        for key, val in self.data.items():
            if len(mask) == len(self.data[key]):
                self.data[key] = val[mask]
        self.point_count = len(self.data['x'])
    
    def add_dimension(self, dim: str, data: Any) -> None:
        if dim in self.point_dimensions:
            raise ValueError(f'Dimension {dim} already exists in las file')
        self.point_dimensions.append(dim)
        self.data[dim] = data

    def __getattr__(self, name: str) -> Any:
        if name not in self.point_dimensions:
            raise AttributeError(f'Attribute \'{name}\' does not exist')
        return self.data[name]

In [5]:
# las = LasPointData(las_files[1])
las = LasPointData(las_file)
las.point_count

92842435

In [6]:
# features = ['classification', 'intensity', 'return_number', 'gps_time', 'x', 'y', 'z']
# npdata = np.hstack([las.data[f].reshape(-1, 1) for f in features])
# df = pd.DataFrame(npdata, columns=features)
# df.classification = df.classification.map(lambda x: 1 if x == 6 else -1)

# df[features[1:]].corrwith(df.classification.astype('float'), method=pointbiserialr)

### Display some info about the LiDAR data

In [7]:
# Display header information
las.header_info()

are_points_compressed: False
creation_date: 2023-02-25
evlrs: []
extra_header_bytes: b''
extra_vlr_bytes: b''
file_source_id: 0
generating_software: ArcGIS
global_encoding: <laspy.header.GlobalEncoding object at 0x31bf7d910>
major_version: 1
maxs: [4.83999999e+05 5.45800000e+06 7.48853000e+02]
minor_version: 4
mins: [4.8300e+05 5.4570e+06 1.4573e+01]
number_of_evlrs: 0
number_of_points_by_return: [40879944 25033837 15198490  7432236  2974999   980090   267791    61240
    11732     1824      221       30        1        0        0]
offset_to_point_data: 1003
offsets: [     -0. 7000000.      -0.]
point_count: 92842435
point_format: <PointFormat(7, 0 bytes of extra dims)>
scales: [0.001 0.001 0.001]
start_of_first_evlr: 0
start_of_waveform_data_packet_record: 0
system_identifier: CONVERSION
uuid: 00000000-0000-0000-0000-000000000000
version: 1.4
vlrs: [<VLR(user_id: 'ESRI_Export', record_id: '5', data len: 8)>, <VLR(user_id: 'ESRI_Export', record_id: '8', data len: 8)>, <laspy.vlrs.known

In [8]:
# Display the CRS
las.crs()

<Projected CRS: EPSG:26910>
Name: NAD83 / UTM zone 10N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: UTM zone 10N
- method: Transverse Mercator
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich



In [9]:
# Display the point format dimensions
las.get_point_format()

(<PointFormat(7, 0 bytes of extra dims)>,
 ['x',
  'y',
  'z',
  'X',
  'Y',
  'Z',
  'intensity',
  'return_number',
  'number_of_returns',
  'synthetic',
  'key_point',
  'withheld',
  'overlap',
  'scanner_channel',
  'scan_direction_flag',
  'edge_of_flight_line',
  'classification',
  'user_data',
  'scan_angle',
  'point_source_id',
  'gps_time',
  'red',
  'green',
  'blue'])

### Apply some masks to remove unwanted data

In [10]:
# # This cell naively downsamples the data by selecting only every kth point
# # This was used for testing below algos with lower computational cost before running the whole dataset

# # Create a mask to select only every kth point
# k = 20
# mask = np.zeros(las.point_count, dtype=bool)
# mask[::k] = True

# las.apply_mask(mask)
# print(f'{np.sum(~mask)}/{len(mask)} points removed')
# del mask

In [11]:
# This cell removes points that are labelled as noise in the las file.

# Remove labeled noise
# 7 is the LIDAR classification code for noise
noise_mask = np.ones(las.point_count, dtype=bool)
noise_mask[las.classification == 7] = False

las.apply_mask(noise_mask)
print(f'{np.sum(~noise_mask)}/{len(noise_mask)} points removed')
del noise_mask

23154/92842435 points removed


In [12]:
# This cell removes points that are labelled as never classified or unclassified in the las file.

# Remove labeled noise
# 0 is the LIDAR classification code for never assigned
# 1 is the LIDAR classification code for unassigned
unclassified_mask = np.ones(las.point_count, dtype=bool)
unclassified_mask[(las.classification == 0) | (las.classification == 1)] = False

las.apply_mask(unclassified_mask)
print(f'{np.sum(~unclassified_mask)}/{len(unclassified_mask)} points removed')
del unclassified_mask

6302188/92819281 points removed


In [13]:
# # This cell is an alternative noise removal method that removes noise though statistical outlier rejection

# # Use statistical outlier removal to remove noise
# point_data = las.get_point_data()
# kdtree = KDTree(point_data)  # Use a KDTree for fast nearest neighbour search

# k = 20  # Number of neighbors
# z_thresh = 15.0  # Z-score threshold for outliers

# # Compute distances to k nearest neighbors
# dists, _ = kdtree.query(point_data, k=k + 1)  # k + 1 because query includes the point itself
# z_scores = np.abs((dists.mean(axis=1) - dists.mean()) / dists.std())

# del kdtree, dists

# # Filter points
# sor_mask = z_scores < z_thresh
# las.apply_mask(sor_mask)

# print(f'{np.sum(~sor_mask)}/{len(sor_mask)} points removed')

In [14]:
# Crop the region to 1/4 of its original area

point_data = las.get_point_data()

xmin = min(point_data[:, 0])
ymin = min(point_data[:, 1])
xmax = max(point_data[:, 0])
ymax = max(point_data[:, 1])

xrange = max(point_data[:, 0]) - xmin
yrange = max(point_data[:, 1]) - ymin

if 'kits' in str(data_folder):
    xmax = xmin + xrange / 2.0
    ymax = ymin + yrange / 2.0
else:
    xmin = xmax - xrange / 2.0
    ymin = ymax - xrange / 2.0

range_mask = np.zeros(las.point_count, dtype=bool)
range_mask[(point_data[:, 0] <= xmax) & (point_data[:, 0] >= xmin) & (point_data[:, 1] < ymax) & (point_data[:, 1] >= ymin)] = True

las.apply_mask(range_mask)
print(f'{np.sum(~range_mask)}/{len(range_mask)} points removed')
del range_mask, point_data

66026771/86517093 points removed


### Classify unclassified points

In [15]:
# This cell reclassifies all unclassified points using nearest neighbour classification
# 1 is the LIDAR classification code for unclassified points

# Reclassify unclassified points using nearest neighbour classification
point_data = las.get_point_data()

# Separate classified and unclassified points
classified_mask = las.classification != 1
classified_points = point_data[classified_mask]
classified_classes = las.classification[classified_mask]
unclassified_points = point_data[~classified_mask]

# Use a KDTree to get the indices of the nearest classified points to each unclassified point
kdtree = KDTree(classified_points)
_, neighbour_indices = kdtree.query(unclassified_points, k=20)

# Get the most common class of the neighbours of each unclassified point
neighbour_classes = classified_classes[neighbour_indices]
most_common_neighbour_classes = scipy_mode(neighbour_classes, axis=1).mode

# Update the classes of the unclassified points
las.classification[~classified_mask] = most_common_neighbour_classes
print(f'{sum(~classified_mask)} unclassified points classified using nearest neighbour interpolation')

# Delete large vairables to conserve memory
del classified_mask, kdtree, classified_points, classified_classes, unclassified_points
del point_data, neighbour_indices, neighbour_classes, most_common_neighbour_classes

0 unclassified points classified using nearest neighbour interpolation


### Compute the height above ground

In [16]:
# # This cell uses a CSF to separate ground points from non-gound points

# # Use a cloth simulation filter to separate ground points from non-ground points
# point_data = las.get_point_data()

# csf = CSF.CSF()

# csf.params.bSloopSmooth = False
# # csf.params.cloth_resolution = 0.5
# # csf.params.rigidness = 3

# # These can be left as default
# # csf.params.time_step = 0.65
# # csf.params.class_threshold = 0.5
# # csf.params.interations = 500

# csf.setPointCloud(point_data)

# ground_indices = CSF.VecInt()  # a list to indicate the index of ground points after calculation
# non_ground_indices = CSF.VecInt() # a list to indicate the index of non-ground points after calculation
# csf.do_filtering(ground_indices, non_ground_indices) # do actual filtering.
# ground_indices = np.asarray(ground_indices)
# non_ground_indices = np.asarray(non_ground_indices)
# ground = point_data[ground_indices]
# non_ground = point_data[non_ground_indices]

# os.remove('cloth_nodes.txt')
# del csf

In [17]:
# This code uses the provided classification to separate ground and non-ground points

point_data = las.get_point_data()
las_classification = las.classification
ground_indices = np.arange(len(las_classification))[las_classification == 2]
non_ground_indices = np.arange(len(las_classification))[las_classification != 2]
ground = point_data[ground_indices]
non_ground = point_data[non_ground_indices]

In [18]:
# Interpolate the ground points to get the ground elevation of the non-ground points

griddata_partial = analysis_utils.GriddataPartial(ground=ground, method='linear')
non_ground_xy_split = np.array_split(non_ground[:, :2], PROCESSES)
with Pool(PROCESSES) as pool:
    non_ground_z_proj_split = pool.map(griddata_partial, non_ground_xy_split)
non_ground_z_proj = np.hstack(non_ground_z_proj_split) # ground elevation of each non-ground point

In [19]:
# Some values of non_ground_z_proj are nan, so we use nearest neighbour interpolation to fill in these values
# This is much faster than the interpolation above since we are using a nearest neighbour interpolation instead of a linear one
griddata_partial = analysis_utils.GriddataPartial(ground=ground, method='nearest')

nan_mask = np.isnan(non_ground_z_proj)
non_ground_xy_giving_nan = non_ground[nan_mask, :2]  # xy points giving nan values with the linear interpolation 

non_ground_z_proj_nan_fixed = griddata_partial(non_ground_xy_giving_nan)  # re-interpolate using a nn approach
non_ground_z_proj[nan_mask] = non_ground_z_proj_nan_fixed

In [20]:
# Compute the height above ground of all points
non_ground_heights = non_ground[:, 2] - non_ground_z_proj  # height above ground of each non-ground point

point_data_normalized_height = point_data.copy()
point_data_normalized_height[ground_indices, 2] = 0
point_data_normalized_height[non_ground_indices, 2] = non_ground_heights

# Delete large variables to conserve memory
del ground_indices, non_ground_indices, ground, non_ground, point_data, griddata_partial
del non_ground_xy_split, non_ground_z_proj_split, non_ground_z_proj, non_ground_heights
del nan_mask, non_ground_xy_giving_nan, non_ground_z_proj_nan_fixed

In [21]:
assert not np.isnan(point_data_normalized_height).any(), 'Some normalized height vals are nan!'

In [22]:
nh_path = data_folder / 'normalized_height_linear.pkl'

In [23]:
# Save the normalized height data to avoid recomputing it later
with open(nh_path, 'wb') as f:
    dill.dump(point_data_normalized_height, f)

In [24]:
# Load the normalized height data
with open(nh_path, 'rb') as f:
    point_data_normalized_height = dill.load(f)
las.add_dimension('z_normalized', point_data_normalized_height[:, 2])

### Compute the height variation

Height variation is the absolute difference between min and max values of normalized height within a disk of radius $r$.

In [25]:
r = 0.5

In [26]:
# Use a kd-tree to compute the height variation of all points

point_data_normalized_height_xy = point_data_normalized_height[:, :2]
kdtree = KDTree(point_data_normalized_height_xy)

# Compute height variation by querying the kd-tree for each point in the point cloud
# Use multiple processes to speed up computation
height_variation_partial = analysis_utils.HeightVariation(point_data_normalized_height, kdtree, r, method='mad')

n = len(point_data_normalized_height)
if PROCESSES == 1:
    index_endpoints = [(0, n)]
else:
    chunk_size = n // PROCESSES
    index_endpoints = [(chunk_size * i, chunk_size * (i + 1)) for i in range(PROCESSES - 1)]
    index_endpoints.append((index_endpoints[-1][1], n))

with Pool(PROCESSES) as pool:
    height_variation_split = pool.map(height_variation_partial, index_endpoints)
height_variation = np.hstack(height_variation_split)

del kdtree, point_data_normalized_height_xy, point_data_normalized_height, height_variation_split

In [27]:
hv_path = data_folder / f'height_variation_{"1m" if r == 1.0 else f"{int(r * 100)}cm"}.pkl'

In [28]:
# Save the height_variation data to avoid recomputing it later
with open(hv_path, 'wb') as f:
    dill.dump(height_variation, f)

In [29]:
# Load the height variation
with open(hv_path, 'rb') as f:
    height_variation = dill.load(f)
las.add_dimension('height_variation', height_variation)

### Compute the normal variation

Normal variation is the negative of the average dot product of each normal with other normals within a disk of radius $r$. This value gives a measure of planarity near each point.

In [30]:
# Use open3d to estimate surface normals

point_data = las.get_point_data(normalized=True)
geom = o3d.geometry.PointCloud()
geom.points = o3d.utility.Vector3dVector(point_data)

k = 100
geom.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=k))
direction = np.array([1.0, 0.0, 1.0]) if 'downtown' in str(data_folder) else np.ones(3, dtype=float)
geom.orient_normals_to_align_with_direction(direction)
# geom.orient_normals_consistent_tangent_plane(k=k)  # RuntimeError
normals = np.asarray(geom.normals)

del geom

In [31]:
r = 0.5

In [32]:
# Compute the normal variation

point_data_xy = point_data[:, :2]
kdtree = KDTree(point_data_xy)

normal_variation_partial = analysis_utils.NormalVariation(point_data_xy, normals, kdtree, r, use_tqdm=True)
n = len(point_data_xy)

# if PROCESSES == 1:
#     index_endpoints = [(0, n)]
# else:
#     chunk_size = n // PROCESSES
#     index_endpoints = [(chunk_size * i, chunk_size * (i + 1)) for i in range(PROCESSES - 1)]
#     index_endpoints.append((index_endpoints[-1][1], n))

# with Pool(PROCESSES) as pool:
#     normal_variation_split = pool.map(normal_variation_partial, index_endpoints)
# normal_variation = np.hstack(normal_variation_split)

normal_variation = normal_variation_partial((0, n))

del kdtree, point_data_xy, point_data, normals

100%|██████████| 20490322/20490322 [05:26<00:00, 62792.42it/s]


In [33]:
nv_path = data_folder / f'normal_variation_{"1m" if r == 1.0 else f"{int(r * 100)}cm"}.pkl'

In [34]:
# Save the normal variation data to avoid recomputing it later
with open(nv_path, 'wb') as f:
    dill.dump(normal_variation, f)

In [35]:
# Load the normal variation
with open(nv_path, 'rb') as f:
    normal_variation = dill.load(f)
las.add_dimension('normal_variation', normal_variation)

### Downsample the point cloud and associated data using voxel downsampling

In [36]:
# Create a dictionary with each entry corresponding to one voxel

points = las.get_point_data(normalized=True)
voxel_size = 1.0  # One point per cubic meter
voxel_indices = np.floor(points / voxel_size).astype(int)

# Use a dictionary to store the sum of points and counts for each voxel
voxel_dict = {}
for voxel, point, hv, nv, its, cls in tqdm(zip(voxel_indices, points, height_variation, normal_variation, las.intensity.astype(np.uint64), las.classification), total=len(points)):
    voxel_key = tuple(voxel)
    if voxel_key in voxel_dict:
        voxel_dict[voxel_key][0] += point
        voxel_dict[voxel_key][1] += hv
        voxel_dict[voxel_key][2] += nv
        voxel_dict[voxel_key][3] += its
        voxel_dict[voxel_key][4].append(cls)
        voxel_dict[voxel_key][5] += 1
    else:
        voxel_dict[voxel_key] = [point, hv, nv, its, [cls], 1]

100%|██████████| 20490322/20490322 [00:49<00:00, 418012.36it/s]


In [37]:
# Compute the average for each voxel and create the downsampled point cloud
downsampled_points = np.empty((len(voxel_dict), 7))
for i, ((x, y, z), hv, nv, its, cls, k) in tqdm(enumerate(voxel_dict.values()), total=len(voxel_dict)):
    downsampled_points[i] = np.array([x / k, y / k, z / k, hv / k, nv / k, its / k, stats_mode(cls)])

del voxel_dict

100%|██████████| 2176814/2176814 [00:10<00:00, 207070.30it/s]


In [38]:
# Create a pandas dataframe containing the data of interest from each point in the downsampled point cloud and save it as a csv

columns = ['x', 'y', 'z', 'height_variation', 'normal_variation', 'intensity', 'classification']
df = pd.DataFrame(downsampled_points, columns=columns)

df['classification'] = df['classification'].astype(int)
df[['x', 'y', 'z', 'intensity']] = df[['x', 'y', 'z', 'intensity']].round(2)
df[['height_variation', 'normal_variation']] = df[['height_variation', 'normal_variation']].round(4)

df.to_csv(data_folder / f'{"1m" if voxel_size == 1.0 else f"{int(voxel_size*100)}cm"}_lidar.csv', index=False)

In [39]:
df.head()

,x,y,z,height_variation,normal_variation,intensity,classification
0,483500.53,5457500.47,8.58,2.9096,-0.4244,46.62,5
1,483537.32,5457500.31,6.44,0.9901,-0.3056,66.75,5
2,483567.37,5457500.48,32.16,3.1157,-0.7190,341.96,5
3,483596.23,5457500.09,43.17,2.3543,-0.0939,136.60,5
4,483585.47,5457500.00,39.73,3.6597,-0.6882,164.00,5


In [42]:
df = pd.read_csv(data_folder / '1m_lidar.csv')

In [43]:
# Visualization

# points = las.get_point_data()
# points[:, 2] = 0
# points = downsampled_points[:, :3]  
points = df[['x','y','z']].to_numpy()  # The downsampled, processed point cloud

geom = o3d.geometry.PointCloud()
geom.points = o3d.utility.Vector3dVector(points)

# Use a colormap from Matplotlib
cmap = plt.get_cmap('viridis')

# Map integers to colors using the colormap
# colour_var = np.asarray(las.height_variation)
colour_var = np.asarray(df.height_variation)
colour_var_norm = (colour_var - colour_var.min()) / (colour_var.max() - colour_var.min())
colors = cmap(colour_var_norm)[:, :3]  # [:3] to exclude the alpha channel
geom.colors = o3d.utility.Vector3dVector(colors)

o3d.visualization.draw_geometries([geom])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
